In [7]:
from saber.saber import Saber, Config
config = Config('C:/Users/User/config.ini')
sb = Saber(config)

Using TensorFlow backend.


Saber version: 0.1.0-alpha


In [8]:
sb.load_dataset('C:/Users/User') 

Loading (single) dataset... Done (163.52 seconds).


In [0]:
sb.build(model_name='MT-LSTM-CRF')

Building the multi-task BiLSTM-CRF model... Done (53.60 seconds).


In [0]:
sb.train()

Using train/test/valid strategy...
Global epoch: 1/50
--------------------
Train on 4831 samples, validate on 537 samples
Epoch 1/1
4831/4831 [==============================] - ETA: 3:30:19 - loss: 20.677 - ETA: 2:04:55 - loss: 20.337 - ETA: 1:34:14 - loss: 20.506 - ETA: 1:16:25 - loss: 23.347 - ETA: 1:05:28 - loss: 21.755 - ETA: 56:36 - loss: 21.6409  - ETA: 50:18 - loss: 20.342 - ETA: 46:33 - loss: 20.313 - ETA: 42:49 - loss: 20.009 - ETA: 39:42 - loss: 20.503 - ETA: 37:14 - loss: 20.184 - ETA: 35:47 - loss: 19.980 - ETA: 33:57 - loss: 19.267 - ETA: 32:21 - loss: 18.980 - ETA: 31:35 - loss: 18.953 - ETA: 31:01 - loss: 18.733 - ETA: 30:21 - loss: 18.733 - ETA: 29:46 - loss: 18.710 - ETA: 28:43 - loss: 18.704 - ETA: 27:45 - loss: 18.511 - ETA: 26:55 - loss: 18.539 - ETA: 26:15 - loss: 18.465 - ETA: 25:36 - loss: 18.699 - ETA: 24:57 - loss: 18.614 - ETA: 24:21 - loss: 18.462 - ETA: 23:47 - loss: 18.709 - ETA: 23:15 - loss: 18.613 - ETA: 22:46 - loss: 18.379 - ETA: 22:17 - loss: 18.352 -

In [0]:
sb.save('C:/Users/User/output')

Saving model... Done (496.69 seconds).
Model was saved to C:\Users\User\output


In [0]:
del sb
saber = Saber()
saber.load('C:/Users/User/output')

Loading model... Unzipping... Done (627.85 seconds).


In [1]:
from nltk.corpus.reader.conll import ConllCorpusReader
conll_parser = ConllCorpusReader('C:/Users/User/Desktop/al/учёба/course', '.csv', ('words', 'pos'))
sents = conll_parser.tagged_sents('test.csv')
sents

[[('I', 'O'), ('feel', 'O'), ('a', 'O'), ('bit', 'B-Disease'), ('drowsy', 'I-Disease'), ('&', 'O'), ('have', 'O'), ('a', 'O'), ('little', 'B-Disease'), ('blurred', 'I-Disease'), ('vision', 'I-Disease'), (',', 'O'), ('so', 'O'), ('far', 'O'), ('no', 'O'), ('gastric', 'B-Disease'), ('problems', 'I-Disease'), ('.', 'O')], [('I', 'O'), ("'", 'O'), ('ve', 'O'), ('been', 'O'), ('on', 'O'), ('Arthrotec', 'B-Drug'), ('50', 'O'), ('for', 'O'), ('over', 'O'), ('10', 'O'), ('years', 'O'), ('on', 'O'), ('and', 'O'), ('off', 'O'), (',', 'O'), ('only', 'O'), ('taking', 'O'), ('it', 'O'), ('when', 'O'), ('I', 'O'), ('needed', 'O'), ('it', 'O'), ('.', 'O')], ...]

In [2]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
result = []
for i in conll_parser.sents('test.csv'):
    result.append(TreebankWordDetokenizer().detokenize(i))

In [6]:
sb.annotate('I feel a bit drowsy & have a little blurred vision , so far no gastric problems')

Loading preprocessor... Done (65.78 seconds).


{'text': 'I feel a bit drowsy & have a little blurred vision , so far no gastric problems',
 'title': '',
 'ents': [{'start': 13, 'end': 19, 'text': 'drowsy', 'label': 'Disease'},
  {'start': 36, 'end': 50, 'text': 'blurred vision', 'label': 'Disease'}]}

In [7]:
with open('first_result.txt', 'w') as first_out:
    for i in result:
        first_out.write(str(sb.annotate(i)))
        first_out.write('\n')

In [10]:
def make_new_sents(sentences):
    new_sentences = []
    part = ''
    part_drug=''
    for sent in sentences:
        new_sent = []    
        for elem in sent:
            if elem[1]=='B-Disease':
                part = elem[0]
            elif elem[1]=='I-Disease':
                part = part +' '+ elem[0]
            elif elem[1]=='B-Drug':
                part_drug = elem[0]
            elif elem[1]=='I-Drug':
                part_drug = part_drug +' '+ elem[0]
            else:
                if part:
                    new_sent.append((part, 'Disease'))
                    #new_sent.append(part)
                    part = ''
                if part_drug:
                    new_sent.append((part_drug, 'Drug'))
                    part_drug=''
                #else:    
                #    new_sent.append(elem)
        new_sentences.append(new_sent)    
    return new_sentences    

In [11]:
def print_predictions(filename, sentences):
    import ast
    with open('correct_pred.txt', 'w') as correct, open('partly_correct.txt', 'w') as partly, open('false_pred.txt', 'w') as fal:
        with open(filename) as predicted:
            for line, sent in zip(predicted, sentences):
                line_as_dict = ast.literal_eval(line)
                entities = line_as_dict['ents']
                for ent in entities:  
                    flag = False
                    for elem in sent:
                        if (ent['text'] in elem[0] or elem[0] in ent['text']):
                            flag = True
                            if ent['text'] != elem[0]:
                                partly.write(str(ent['text']+'\t'+elem[0]+'\n'))
                            else:
                                correct.write(str(line_as_dict['text']+'\n'+ent['text']+'\t'+ent['label']+'\n\n'))    
                    if not flag:
                        fal.write(str(line_as_dict['text']+'\n'+ent['text']+'\t'+ent['label']+'\n\n'))  
    return                    

In [13]:
new_sents = make_new_sents(sents)
print_predictions('first_result.txt', new_sents)

In [6]:
with open('training_res.txt', 'w') as out:
    for i in range(9):
        file = 'C:/Users/User/output/User/train_session_sat_oct_19_12_22_17/'+f'epoch_00{i+1}.txt'
        with open(file) as infile:
            for line in infile:
                out.write(line)
    for i in range(9, 49):
        file = 'C:/Users/User/output/User/train_session_sat_oct_19_12_22_17/'+f'epoch_0{i+1}.txt'
        with open(file) as infile:
            for line in infile:
                out.write(line)